In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from openai import OpenAI
import pandas as pd
import numpy as np
import getpass
import openai
import math
import time
import json
import os

In [2]:
#Key
os.environ['OPENAI_API_KEY'] = "keygi"

Đọc data

In [3]:
text = []
with open('D:\\demo\\test\\sents.txt', 'r', encoding='utf-8') as file:
    for line in file:
        text.append(line.strip())

label = []
with open('D:\\demo\\test\\sentiments.txt', 'r', encoding='utf-8') as file:
    for line in file:
        label.append(line.strip())

df = pd.DataFrame({'text': text, 'label': label})
df

,text,label
0,nói tiếng anh lưu loát .,2
1,giáo viên rất vui tính .,2
2,cô max có tâm .,2
3,"giảng bài thu hút , dí dỏm .",2
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...",0
...,...,...
3161,"các slide khó hiểu , ngôn ngữ trong slide phức...",0
3162,giáo viên giảng dạy có tâm huyết .,2
3163,chia sẻ cho em nhiều điều hay .,2
3164,em tiếp thu chậm .,0


Bỏ neutral

In [5]:
df = df[df['label'] != '1'].copy()
for i in df.index:
  if df['label'][i] == '2':
    df['label'][i] = '1'
df.reset_index(drop=True, inplace=True)
df['label'] = df['label'].astype(int)
df

,text,label
0,nói tiếng anh lưu loát .,1
1,giáo viên rất vui tính .,1
2,cô max có tâm .,1
3,"giảng bài thu hút , dí dỏm .",1
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...",0
...,...,...
2994,"giảng viên rất nhiệt tình , công bằng trong vi...",1
2995,"các slide khó hiểu , ngôn ngữ trong slide phức...",0
2996,giáo viên giảng dạy có tâm huyết .,1
2997,chia sẻ cho em nhiều điều hay .,1


df_label_0 = df[df['label'] == 0]
df_label_1 = df[df['label'] == 1]

df_sampled_0 = df_label_0.sample(n=300, random_state=42)  # random_state để đảm bảo tính tái lập
df_sampled_1 = df_label_1.sample(n=300, random_state=42)

df_sampled = pd.concat([df_sampled_0, df_sampled_1])

df_sampled = df_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

df_sampled.to_csv('/content/drive/MyDrive/SA/Dataset600.csv', index=False)

In [ ]:
output_path = r"D:\\demo\\Dataset_clear.csv"
df.to_csv(output_path, index=False)

In [8]:
df = pd.read_csv('D:\\demo\\Dataset_clear.csv')
df['ID'] = range(0, len(df) )
df

,text,label,ID
0,nói tiếng anh lưu loát .,1,0
1,giáo viên rất vui tính .,1,1
2,cô max có tâm .,1,2
3,"giảng bài thu hút , dí dỏm .",1,3
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...",0,4
...,...,...,...
2994,"giảng viên rất nhiệt tình , công bằng trong vi...",1,2994
2995,"các slide khó hiểu , ngôn ngữ trong slide phức...",0,2995
2996,giáo viên giảng dạy có tâm huyết .,1,2996
2997,chia sẻ cho em nhiều điều hay .,1,2997


In [9]:
print(df.text.head(20))

0                              nói tiếng anh lưu loát .
1                              giáo viên rất vui tính .
2                                       cô max có tâm .
3                          giảng bài thu hút , dí dỏm .
4     giáo viên không giảng dạy kiến thức , hướng dẫ...
5                    thầy dạy nhiệt tình và tâm huyết .
6                          tính điểm thi đua các nhóm .
7              thầy nhiệt tình giảng lại cho học sinh .
8     có đôi lúc nói hơi nhanh làm sinh viên không t...
9     giảng dạy nhiệt tình , liên hệ thực tế khá nhi...
10     giảng viên nhiệt tình trong công tác giảng dạy .
11        cô rất nhiệt tình , dễ thương , dạy dễ hiểu .
12    trong trường macbook thầy số hai thì không có ...
13    sinh viên không tiếp thu kịp cũng như không hi...
14    thầy nhiệt tình giúp đỡ sinh viên trong quá tr...
15    thầy rất tận tình và giúp đỡ sinh viên rất nhi...
16               giảng viên giải thích kỹ và chi tiết .
17           thầy dạy rất chi tiết , lý thuyết đ

### **Các mẫu Prompt**

In [10]:
def zero_shot_prompt(formatted_sentences):
  return f"""
  Bạn là một chuyên gia trong lĩnh vực Phân tích cảm xúc văn bản, cụ thể trong các câu đánh giá trường đại học. Hãy giúp tôi phân tích các văn bản sau thành tiêu cực và tích cực:

  '
  {formatted_sentences}
  '

  Chỉ phân tích 2 cảm xúc là 'P' cho Tích cực (Positive) và 'N' cho Tiêu cực (Negative). Cùng với ID của câu văn bản.
  Trình bày theo dạng list json với 2 cột là 'ID' : '$ID của câu' và 'S' : '$Sentiment của câu'
  Vui lòng đảm bảo đủ số lượng các mãu theo ID, không được bỏ xót bất kỳ mẫu nào.
  Lưu ý chỉ trả về duy nhất list json, không giải thích hay kèm theo bất kỳ thứ gì
  """


def one_shot_prompt(formatted_sentences):
  return f"""
  Bạn là một chuyên gia trong lĩnh vực Phân tích cảm xúc văn bản, cụ thể trong các câu đánh giá trường đại học. Tôi muốn phân tích các văn bản thành tiêu cực và tích cực:

  '
  {formatted_sentences}
  '

  Chỉ phân tích 2 cảm xúc là 'P' cho Tích cực (Positive) và 'N' cho Tiêu cực (Negative). Cùng với ID của câu văn bản.
  Trả về kết quả theo dạng list json với 2 cột là "ID" : "$ID của câu" và "S" : "$Sentiment của câu"
  Vui lòng đảm bảo đủ số lượng các mãu theo ID, không được bỏ xót bất kỳ mẫu nào:

  ví dụ tôi có một mẫu với ID là 30684 và nội dung văn bản là "Ngôi trường này có cơ sở vật chất thật tốt", sau khi phân tích trả về kết quả:

  [
    {"{"}"ID": 30684, "S": "P"{"}"}
  ]

  Lưu ý chỉ trả về duy nhất list json, không giải thích hay kèm theo bất kỳ thứ gì
  """


def few_shot_prompt(formatted_sentences):
  return f"""
  Tôi có các văn bản đánh giá của sinh viên về trường đại học như sau:

  '
  {formatted_sentences}
  '

  Bạn là một chuyên gia trong lĩnh vực Phân tích cảm xúc văn bản, Cụ thể trong các câu đánh giá trường đại học. Tôi muốn phân tích các văn bản thành tiêu cực và tích cực:
  Chỉ phân tích 2 cảm xúc là 'P' cho Tích cực (Positive) và 'N' cho Tiêu cực (Negative). Cùng với ID của câu văn bản.
  Trả về kết quả theo dạng list json với 2 cột là "ID" : "$ID của câu" và "S" : "$Sentiment của câu"
  Hãy phần tích các văn bản, đảm bảo đủ số lượng các mãu theo ID, không được bỏ xót bất kỳ mẫu nào.

  Sau đây là một số ví dụ, có các mẫu dữ liệu gồm ID và Nội dung như sau:

  30684 : "Giáo viên ở đây rất tốt, dạy rất nhiệt tình",
  30685 : "Nhà vệ sinh tệ, bốc mùi",
  30686 : "Tôi không thích cơ sở vật chất ở nơi này, nó cũ kĩ",
  30687 : "Không gian trường học thoáng mát, nhiều cây xanh đẹp".

  Sau khi phân tích trả về kết quả list json như sau:

  [
    {"{"}"ID": 30684, "S": "P"{"}"},
    {"{"}"ID": 30685, "S": "N"{"}"},
    {"{"}"ID": 30686, "S": "N"{"}"},
    {"{"}"ID": 30687, "S": "P"{"}"},
  ]

  Lưu ý chỉ trả về duy nhất list json, không giải thích kèm theo tên list hay bất kỳ thứ gì
  """


def cot_prompt(formatted_sentences):
    return f"""
    Tôi có các văn bản đánh giá của sinh viên về trường đại học như sau:

    '
    {formatted_sentences}
    '

    Bạn là một chuyên gia phân loại cảm xúc trong ngữ cảnh trường đại học. Bạn cần phân loại tích cực (P) hay tiêu cực (N).
    Bạn phải trả về kiểu list json gồm 2 cột "ID": "$ID của câu" và "S": "$Sentiment của câu" và không được trả về các bước suy luận.

    Ta có một mẫu đánh giá có ID: 1234 - Nội dung: thầy vui tính, hòa đồng.
    Bước 1: Câu này đang nói tới khía cạnh tính cách của thầy giáo
    Bước 2: Thầy giáo có tính cách vui tính hòa đồng là cảm xúc tích cực
    Bước 3: Cảm xúc của cả câu là tích cực
    Kết quả: [{"{"}"ID": "1234", "S": "P"{"}"}]

    Chú ý, chỉ trả về list json, tuyệt đối không không giải thích hay nói gì thêm.
    Nhắc lại kết quả trả về mong đợi chỉ là list json, không suy luận hay kèm thêm thứ gì.
    """




def zero_cot_prompt(formatted_sentences):
  return f"""
  Tôi có các văn bản đánh giá của sinh viên về trường đại học như sau:

    '
    {formatted_sentences}
    '

  Bạn là một chuyên gia trong lĩnh vực Phân tích cảm xúc văn bản, cụ thể trong các câu đánh giá trường đại học. Tôi muốn phân tích các văn bản thành tiêu cực và tích cực:
  HÃY SUY LUẬN TỪNG BƯỚC theo góc độ xử lý ngôn ngữ tự nhiên. Ngữ cảnh chính ở đây là đánh giá của sinh viên về trường đại học.
  Chỉ phân tích 2 cảm xúc là 'P' cho Tích cực (Positive) và 'N' cho Tiêu cực (Negative). Cùng với ID của câu.
  Kết quả thu về dưới dạng list json gồm 2 cột là "ID" : "$ID của câu" và "S" : "$Sentiment của câu".
  Chỉ trả về kết quả là list json có 2 cột "ID" và "S" duy nhất, không giải thích nêu tên list hay nói gì thêm.
  """

### **Truy cập api để SA**

In [11]:
def analyze_sentiments(df, batch_size, llm, prompt_template):
    sentiment_results = []
    num_batches = math.ceil(len(df) / batch_size)

    for i in range(num_batches):
        # Lấy batch dữ liệu
        batch_df = df.iloc[i * batch_size:(i + 1) * batch_size]
        # Tạo chuỗi văn bản với ID và câu
        formatted_sentences = "\n".join([f"{row['ID']}: \"{row['text']}\"" for _, row in batch_df.iterrows()])

        # Tạo prompt với định dạng chính xác
        prompt = prompt_template(formatted_sentences)

        # Ghi nhật ký để kiểm tra batch hiện tại
        print(f"Processing batch {i+1}/{num_batches}")

        try:
            # Tạo kết nối với OpenAI
            client = openai.OpenAI()
            response = client.chat.completions.create(
                model=llm,
                messages=[
                    {
                        "role": "system",
                        "content": prompt
                    }
                ],
                temperature=1,
                max_tokens=4096,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
            )

            response_text = response.choices[0].message.content.replace("```json", "").replace("```", "")
            print(f"Raw response text: {response_text}")  # Ghi nhật ký phản hồi để kiểm tra

            # Xử lý JSON từ phản hồi
            response_text = response_text.replace("```json", "").replace("```", "").strip()

            # Chuyển đổi phản hồi thành danh sách từ điển
            try:
                response_data = json.loads(response_text)
                if isinstance(response_data, list):
                    sentiment_results.extend(response_data)
                else:
                    print("Unexpected response format:", response_data)
            except json.JSONDecodeError as e:
                print("Error decoding JSON:", e)

        except Exception as e:
            print(f"Error processing batch {i+1}: {e}")
            continue

    results = pd.DataFrame(sentiment_results)
    return results


Đánh giá

In [12]:
def evaluate_classification(y_pred, y_test):
    mapping = {'P': 1, 'p': 1, 'N': 0, 'n': 0}
    y_pred = y_pred.map(mapping)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # In ra kết quả
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

**Sample**

In [13]:
demo_df = df.head(5)

# **GPT 3.5 turbo**

## **zero-shot learning**

In [14]:
batch_size = 2

llm = "gpt-3.5-turbo"

results_zero_shot = analyze_sentiments(demo_df, batch_size, llm, zero_shot_prompt)

print("Zero-shot results:")

Processing batch 1/3
Raw response text: [
    {"ID": 0, "S": "P"},
    {"ID": 1, "S": "P"}
]
Processing batch 2/3
Raw response text: [
    {"ID": "2", "S": "P"},
    {"ID": "3", "S": "P"}
]
Processing batch 3/3
Raw response text: [
    {"ID": 4, "S": "N"}
]
Zero-shot results:


### **Result zero-shot**

In [23]:
evaluate_classification(results_zero_shot['S'], demo_df['label'])

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


In [17]:
results_zero_shot.to_csv('D:\\demo_code\\results\\GPT 3,5 Turbo\\Zero-shot-learning.csv', index=False)

## **One-shot learning**

In [26]:
batch_size = 2

llm = "gpt-3.5-turbo"

results_one_shot = analyze_sentiments(demo_df, batch_size, llm, one_shot_prompt)
results_one_shot.to_csv('D:\\demo_code\\results\\GPT 3,5 Turbo\\One-shot-learning.csv', index=False)
print("One-shot results:")

Processing batch 1/3
Raw response text: [
    {"ID": 0, "S": "P"},
    {"ID": 1, "S": "P"}
]
Processing batch 2/3
Raw response text: [
    {"ID": 2, "S": "P"},
    {"ID": 3, "S": "P"}
]
Processing batch 3/3
Raw response text: [
    {"ID": 4, "S": "N"}
]
One-shot results:


### **Result one-shot**

In [28]:
evaluate_classification(results_one_shot['S'], demo_df['label'])

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


## few-shot learning

In [31]:
batch_size = 2

llm = "gpt-3.5-turbo"

results_few_shot = analyze_sentiments(demo_df, batch_size, llm, few_shot_prompt)
results_few_shot.to_csv('D:\\demo_code\\results\\GPT 3,5 Turbo\\Few-shot-learning.csv', index=False)
print("Few-shot results:")
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/3
Raw response text: [
    {"ID": 0, "S": "P"},
    {"ID": 1, "S": "P"}
]
Processing batch 2/3
Raw response text: [
    {"ID": 2, "S": "P"},
    {"ID": 3, "S": "P"}
]
Processing batch 3/3
Raw response text: [
    {"ID": 4, "S": "N"}
]
Few-shot results:


In [35]:
results_few_shot = pd.read_csv('D:\\demo_code\\results\\GPT 3,5 Turbo\\Few-shot-learning.csv')
results_few_shot

,ID,S
0,0,P
1,1,P
2,2,P
3,3,P
4,4,N


### **Result few-shot**

In [32]:
evaluate_classification(results_few_shot['S'], demo_df['label'])

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


## **COT**

In [ ]:
batch_size = 2

llm = "gpt-3.5-turbo"

results_cot = analyze_sentiments(df, batch_size, llm, cot_prompt)
results_cot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 3.5 Turbo/cot.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
    {"ID": "0", "S": "P"},
    {"ID": "1", "S": "N"}
]
Processing batch 2/300
Raw response text: [
    {"ID": "2", "S": "N"},
    {"ID": "3", "S": "P"}
]
Processing batch 3/300
Raw response text: [
    {"ID": "4", "S": "P"},
    {"ID": "5", "S": "P"}
]
Processing batch 4/300
Raw response text: [
    {"ID": "6", "S": "P"},
    {"ID": "7", "S": "P"}
]
Processing batch 5/300
Raw response text: [
    {"ID": "8", "S": "P"},
    {"ID": "9", "S": "N"}
]
Processing batch 6/300
Raw response text: [
    {"ID": "10", "S": "N"},
    {"ID": "11", "S": "P"}
]
Processing batch 7/300
Raw response text: [
    {"ID": "12", "S": "P"},
    {"ID": "13", "S": "N"}
]
Processing batch 8/300
Raw response text: [
    {"ID": "14", "S": "P"},
    {"ID": "15", "S": "P"}
]
Processing batch 9/300
Raw response text: [{"ID": "16", "S": "N"}, {"ID": "17", "S": "P"}]
Processing batch 10/300
Raw response text: [
    {"ID": "18", "S": "N"},
    {"ID": "19", "S": "N"}
]
Processin

In [ ]:
evaluate_classification(results_cot['S'], df['label'])

Accuracy: 0.9083
Precision: 0.9140
Recall: 0.9083
F1 Score: 0.9080


Zero - CoT

In [ ]:
batch_size = 2

llm = "gpt-3.5-turbo"

results_zero_cot = analyze_sentiments(df, batch_size, llm, zero_cot_prompt)
results_zero_cot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 3.5 Turbo/Zero_cot.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
    {"ID": "0", "S": "P"},
    {"ID": "1", "S": "N"}
]
Processing batch 2/300
Raw response text: [
    {"ID": "2", "S": "N"},
    {"ID": "3", "S": "P"}
]
Processing batch 3/300
Raw response text: [
    {"ID": "4", "S": "P"},
    {"ID": "5", "S": "P"}
]
Processing batch 4/300
Raw response text: [
    {"ID": "6", "S": "P"},
    {"ID": "7", "S": "P"}
]
Processing batch 5/300
Raw response text: [
    {"ID": "8", "S": "P"},
    {"ID": "9", "S": "N"}
]
Processing batch 6/300
Raw response text: [
    {"ID": "10", "S": "N"},
    {"ID": "11", "S": "P"}
]
Processing batch 7/300
Raw response text: [
    {"ID": "12", "S": "P"},
    {"ID": "13", "S": "N"}
]
Processing batch 8/300
Raw response text: [
    {"ID": "14", "S": "P"},
    {"ID": "15", "S": "P"}
]
Processing batch 9/300
Raw response text: [
    {"ID": "16", "S": "N"},
    {"ID": "17", "S": "P"}
]
Processing batch 10/300
Raw response text: [
    {"ID": "18", "S": "N"},
    {"ID": "19", "S": "N"}
]

### **Result COT**

In [ ]:
evaluate_classification(results_zero_cot['S'], df['label'])

Accuracy: 0.8983
Precision: 0.9045
Recall: 0.8983
F1 Score: 0.8979


# **GPT 4o**

## **Zero shot learning**

In [ ]:
batch_size = 2

llm = "gpt-4o"

results_zero_shot = analyze_sentiments(df, batch_size, llm, zero_shot_prompt)
results_zero_shot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o/Zero-shot-learning.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
    {"ID": 0, "S": "N"},
    {"ID": 1, "S": "N"}
]
Processing batch 2/300
Raw response text: [
  {"ID": 2, "S": "N"},
  {"ID": 3, "S": "P"}
]
Processing batch 3/300
Raw response text: 
[
  {"ID": 4, "S": "P"},
  {"ID": 5, "S": "P"}
]

Processing batch 4/300
Raw response text: [
  {"ID": 6, "S": "N"},
  {"ID": 7, "S": "P"}
]
Processing batch 5/300
Raw response text: [
  {"ID": 8, "S": "P"},
  {"ID": 9, "S": "N"}
]
Processing batch 6/300
Raw response text: [
  {"ID": 10, "S": "N"},
  {"ID": 11, "S": "P"}
]
Processing batch 7/300
Raw response text: [
  {"ID": 12, "S": "P"},
  {"ID": 13, "S": "N"}
]
Processing batch 8/300
Raw response text: [
  {
    "ID": "14",
    "S": "P"
  },
  {
    "ID": "15",
    "S": "P"
  }
]
Processing batch 9/300
Raw response text: [
  {
    "ID": "16",
    "S": "N"
  },
  {
    "ID": "17",
    "S": "P"
  }
]
Processing batch 10/300
Raw response text: [
  {
    "ID": 18,
    "S": "N"
  },
  {
    "ID": 19,
    "S": "N"

### **Result zero-shot**

In [ ]:
evaluate_classification(results_zero_shot['S'], df['label'])

Accuracy: 0.9433
Precision: 0.9443
Recall: 0.9433
F1 Score: 0.9433


## **One shot learning**

In [ ]:
batch_size = 2

llm = "gpt-4o"

results_one_shot = analyze_sentiments(df, batch_size, llm, one_shot_prompt)
results_one_shot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o/One-shot-learning.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
  {"ID": 0, "S": "N"},
  {"ID": 1, "S": "N"}
]
Processing batch 2/300
Raw response text: [
  {"ID": 2, "S": "N"},
  {"ID": 3, "S": "P"}
]
Processing batch 3/300
Raw response text: [
  {"ID": 4, "S": "P"},
  {"ID": 5, "S": "P"}
]
Processing batch 4/300
Raw response text: [
    {"ID": 6, "S": "N"},
    {"ID": 7, "S": "P"}
]
Processing batch 5/300
Raw response text: [
  {"ID": 8, "S": "P"},
  {"ID": 9, "S": "N"}
]
Processing batch 6/300
Raw response text: [
    {"ID": 10, "S": "N"},
    {"ID": 11, "S": "P"}
]
Processing batch 7/300
Raw response text: [
  {"ID": 12, "S": "P"},
  {"ID": 13, "S": "N"}
]
Processing batch 8/300
Raw response text: [
  {"ID": 14, "S": "P"},
  {"ID": 15, "S": "P"}
]
Processing batch 9/300
Raw response text: [
  {"ID": 16, "S": "N"},
  {"ID": 17, "S": "P"}
]
Processing batch 10/300
Raw response text: [
  {"ID": 18, "S": "N"},
  {"ID": 19, "S": "N"}
]
Processing batch 11/300
Raw response text: [
  {"ID": 20, "S": "N"},
  

### **Result one-shot**

In [ ]:
evaluate_classification(results_one_shot['S'], df['label'])

Accuracy: 0.9450
Precision: 0.9458
Recall: 0.9450
F1 Score: 0.9450


## **Few Shot**

In [ ]:
batch_size = 2

llm = "gpt-4o"

results_few_shot = analyze_sentiments(df, batch_size, llm, few_shot_prompt)
results_few_shot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o/few-shot-learning.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
  {"ID": 0, "S": "N"},
  {"ID": 1, "S": "N"}
]
Processing batch 2/300
Raw response text: [
  {"ID": 2, "S": "N"},
  {"ID": 3, "S": "P"}
]
Processing batch 3/300
Raw response text: [
  {"ID": 4, "S": "P"},
  {"ID": 5, "S": "P"}
]
Processing batch 4/300
Raw response text: [
    {"ID": 6, "S": "N"},
    {"ID": 7, "S": "P"}
]
Processing batch 5/300
Raw response text: [
  {"ID": 8, "S": "P"},
  {"ID": 9, "S": "N"}
]
Processing batch 6/300
Raw response text: [
  {"ID": 10, "S": "N"},
  {"ID": 11, "S": "P"}
]
Processing batch 7/300
Raw response text: [
    {"ID": 12, "S": "P"},
    {"ID": 13, "S": "N"}
]
Processing batch 8/300
Raw response text: [
    {"ID": 14, "S": "P"},
    {"ID": 15, "S": "P"}
]
Processing batch 9/300
Raw response text: [
  {"ID": 16, "S": "N"},
  {"ID": 17, "S": "P"}
]
Processing batch 10/300
Raw response text: [
    {"ID": 18, "S": "N"},
    {"ID": 19, "S": "N"}
]
Processing batch 11/300
Raw response text: [
  {"ID": 20, "S": 

### **Result few-shot**

In [ ]:
evaluate_classification(results_few_shot['S'], df['label'])

Accuracy: 0.9467
Precision: 0.9476
Recall: 0.9467
F1 Score: 0.9466


## **COT**

In [ ]:
batch_size = 2

llm = "gpt-4o"

results_cot = analyze_sentiments(df, batch_size, llm, cot_prompt)
results_cot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o/CoT.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
    {"ID": "0", "S": "N"},
    {"ID": "1", "S": "N"}
]
Processing batch 2/300
Raw response text: [
    {"ID": "2", "S": "N"},
    {"ID": "3", "S": "P"}
]
Processing batch 3/300
Raw response text: [
    {"ID": "4", "S": "P"},
    {"ID": "5", "S": "P"}
]
Processing batch 4/300
Raw response text: [
    {"ID": "6", "S": "N"},
    {"ID": "7", "S": "P"}
]
Processing batch 5/300
Raw response text: [
    {"ID": "8", "S": "P"},
    {"ID": "9", "S": "N"}
]
Processing batch 6/300
Raw response text: [
    {"ID": "10", "S": "N"},
    {"ID": "11", "S": "P"}
]
Processing batch 7/300
Raw response text: [
    {"ID": "12", "S": "P"},
    {"ID": "13", "S": "N"}
]
Processing batch 8/300
Raw response text: [
    {"ID": "14", "S": "P"},
    {"ID": "15", "S": "P"}
]
Processing batch 9/300
Raw response text: [
    {"ID": "16", "S": "N"},
    {"ID": "17", "S": "P"}
]
Processing batch 10/300
Raw response text: [
    {"ID": "18", "S": "N"},
    {"ID": "19", "S": "N"}
]

### **Result COT**

In [ ]:
evaluate_classification(results_cot['S'], df['label'])

Accuracy: 0.9500
Precision: 0.9507
Recall: 0.9500
F1 Score: 0.9500


## **zero cot**

In [ ]:
batch_size = 2

llm = "gpt-4o"

results_zero_cot = analyze_sentiments(df, batch_size, llm, zero_cot_prompt)
results_zero_cot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o/Zero_CoT.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
    {"ID": 0, "S": "P"},
    {"ID": 1, "S": "N"}
]
Processing batch 2/300
Raw response text: [
  {
    "ID": 2,
    "S": "N"
  },
  {
    "ID": 3,
    "S": "P"
  }
]
Processing batch 3/300
Raw response text: [
    {
        "ID": 4,
        "S": "P"
    },
    {
        "ID": 5,
        "S": "P"
    }
]
Processing batch 4/300
Raw response text: [
  {"ID": 6, "S": "P"},
  {"ID": 7, "S": "P"}
]
Processing batch 5/300
Raw response text: [
  {
    "ID": 8,
    "S": "P"
  },
  {
    "ID": 9,
    "S": "N"
  }
]
Processing batch 6/300
Raw response text: [
    { "ID": "10", "S": "N" },
    { "ID": "11", "S": "P" }
]
Processing batch 7/300
Raw response text: [
    {
        "ID": 12,
        "S": "P"
    },
    {
        "ID": 13,
        "S": "N"
    }
]
Processing batch 8/300
Raw response text: [
  {
    "ID": 14,
    "S": "P"
  },
  {
    "ID": 15,
    "S": "P"
  }
]
Processing batch 9/300
Raw response text: [
  {
    "ID": 16,
    "S": "N"
  },
  

### **Result Z-COT**

In [ ]:
evaluate_classification(results_zero_cot['S'], df['label'])


Accuracy: 0.9433
Precision: 0.9446
Recall: 0.9433
F1 Score: 0.9433


# **GPT 4o mini**

## **Zero shot learining**

In [ ]:
batch_size = 2

llm = "gpt-4o-mini"

results_zero_shot = analyze_sentiments(df, batch_size, llm, zero_shot_prompt)
results_zero_shot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o mini/Zero-shot-learning.csv', index=False)
print("Zero-shot results:")
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
  {"ID": "0", "S": "P"},
  {"ID": "1", "S": "N"}
]
Processing batch 2/300
Raw response text: 
[
    {"ID": "2", "S": "N"},
    {"ID": "3", "S": "P"}
]

Processing batch 3/300
Raw response text: 
[
    {"ID": "4", "S": "P"},
    {"ID": "5", "S": "P"}
]

Processing batch 4/300
Raw response text: 
[
    {"ID": "6", "S": "N"},
    {"ID": "7", "S": "P"}
]

Processing batch 5/300
Raw response text: 
[
  {"ID": "8", "S": "P"},
  {"ID": "9", "S": "N"}
]

Processing batch 6/300
Raw response text: 
[
  {"ID": "10", "S": "N"},
  {"ID": "11", "S": "P"}
]

Processing batch 7/300
Raw response text: 
[
  {"ID": "12", "S": "P"},
  {"ID": "13", "S": "N"}
]

Processing batch 8/300
Raw response text: 
[
    {"ID": "14", "S": "P"},
    {"ID": "15", "S": "P"}
]

Processing batch 9/300
Raw response text: 
[
  {"ID": 16, "S": "N"},
  {"ID": 17, "S": "P"}
]

Processing batch 10/300
Raw response text: 
[
    {"ID": "18", "S": "N"},
    {"ID": "19", "S": "N"}
]

Proce

### **Result zero**

In [ ]:
results_zero_shot = pd.read_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o mini/Zero-shot-learning.csv')

evaluate_classification(results_zero_shot['S'], df['label'])

Accuracy: 0.9283
Precision: 0.9318
Recall: 0.9283
F1 Score: 0.9282


## **One shot learning**

In [ ]:
batch_size = 2

llm = "gpt-4o-mini"

results_one_shot = analyze_sentiments(df, batch_size, llm, one_shot_prompt)
results_one_shot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o mini/One-shot-learning.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
  {"ID": 0, "S": "P"},
  {"ID": 1, "S": "N"}
]
Processing batch 2/300
Raw response text: [
  {"ID": 2, "S": "N"},
  {"ID": 3, "S": "P"}
]
Processing batch 3/300
Raw response text: [
  {"ID": 4, "S": "P"},
  {"ID": 5, "S": "P"}
]
Processing batch 4/300
Raw response text: [
  {"ID": 6, "S": "N"},
  {"ID": 7, "S": "P"}
]
Processing batch 5/300
Raw response text: [
  {"ID": 8, "S": "P"},
  {"ID": 9, "S": "N"}
]
Processing batch 6/300
Raw response text: [
  {"ID": 10, "S": "N"},
  {"ID": 11, "S": "P"}
]
Processing batch 7/300
Raw response text: [
  {"ID": 12, "S": "P"},
  {"ID": 13, "S": "N"}
]
Processing batch 8/300
Raw response text: [
  {"ID": 14, "S": "P"},
  {"ID": 15, "S": "P"}
]
Processing batch 9/300
Raw response text: [
  {"ID": 16, "S": "N"},
  {"ID": 17, "S": "P"}
]
Processing batch 10/300
Raw response text: [
  {"ID": 18, "S": "N"},
  {"ID": 19, "S": "N"}
]
Processing batch 11/300
Raw response text: [
  {"ID": 20, "S": "N"},
  {"ID": 2

### **Result one**

In [ ]:
evaluate_classification(results_one_shot['S'], df['label'])

Accuracy: 0.9367
Precision: 0.9390
Recall: 0.9367
F1 Score: 0.9366


## **Few-Shot-Learning**

In [ ]:
batch_size = 2

llm = "gpt-4o-mini"

results_few_shot = analyze_sentiments(df, batch_size, llm, few_shot_prompt)
results_few_shot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o mini/Few-shot-learning.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
  {"ID": 0, "S": "P"},
  {"ID": 1, "S": "N"}
]
Processing batch 2/300
Raw response text: [
    {"ID": 2, "S": "N"},
    {"ID": 3, "S": "P"}
]
Processing batch 3/300
Raw response text: [
    {"ID": 4, "S": "P"},
    {"ID": 5, "S": "P"}
]
Processing batch 4/300
Raw response text: [
    {"ID": 6, "S": "N"},
    {"ID": 7, "S": "P"}
]
Processing batch 5/300
Raw response text: [
    {"ID": 8, "S": "P"},
    {"ID": 9, "S": "N"}
]
Processing batch 6/300
Raw response text: [
  {"ID": 10, "S": "N"},
  {"ID": 11, "S": "P"}
]
Processing batch 7/300
Raw response text: [
    {"ID": 12, "S": "P"},
    {"ID": 13, "S": "N"}
]
Processing batch 8/300
Raw response text: [
    {"ID": 14, "S": "P"},
    {"ID": 15, "S": "P"}
]
Processing batch 9/300
Raw response text: [
    {"ID": 16, "S": "N"},
    {"ID": 17, "S": "P"}
]
Processing batch 10/300
Raw response text: [
    {"ID": 18, "S": "N"},
    {"ID": 19, "S": "N"}
]
Processing batch 11/300
Raw response text: [
  

### **Result few**

In [ ]:
results_few_shot = pd.read_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o mini/Few-shot-learning.csv')
evaluate_classification(results_few_shot['S'], df['label'])

Accuracy: 0.9483
Precision: 0.9492
Recall: 0.9483
F1 Score: 0.9483


## **COT**

In [ ]:
batch_size = 2

llm = "gpt-4o-mini"

results_cot = analyze_sentiments(df, batch_size, llm, cot_prompt)
results_cot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o mini/CoT.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [{"ID": "0", "S": "N"},{"ID": "1", "S": "N"}]
Processing batch 2/300
Raw response text: [{"ID": "2", "S": "N"}, {"ID": "3", "S": "P"}]
Processing batch 3/300
Raw response text: [{"ID": "4", "S": "P"}, {"ID": "5", "S": "P"}]
Processing batch 4/300
Raw response text: [{"ID": "6", "S": "N"},{"ID": "7", "S": "P"}]
Processing batch 5/300
Raw response text: [{"ID": "8", "S": "P"}, {"ID": "9", "S": "N"}]
Processing batch 6/300
Raw response text: [{"ID": "10", "S": "N"}, {"ID": "11", "S": "P"}]
Processing batch 7/300
Raw response text: [{"ID": "12", "S": "P"},{"ID": "13", "S": "N"}]
Processing batch 8/300
Raw response text: [{"ID":"14","S":"P"},{"ID":"15","S":"P"}]
Processing batch 9/300
Raw response text: [{"ID": "16", "S": "N"},{"ID": "17", "S": "P"}]
Processing batch 10/300
Raw response text: [{"ID": "18", "S": "N"},{"ID": "19", "S": "N"}]
Processing batch 11/300
Raw response text: [{"ID": "20", "S": "N"},{"ID": "21", "S": "N"}]
Processing batch 12/

### **Result COT**

In [ ]:
evaluate_classification(results_cot['S'], df['label'])

Accuracy: 0.9550
Precision: 0.9556
Recall: 0.9550
F1 Score: 0.9550


## **Zero** COT

In [ ]:
batch_size = 2

llm = "gpt-4o-mini"

results_zero_cot = analyze_sentiments(df, batch_size, llm, zero_cot_prompt)
results_zero_cot.to_csv('/content/drive/MyDrive/SA/Results_SA_20/GPT 4o mini/Zero_CoT.csv', index=False)
#evaluate_classification(results_zero_shot['Sentiment'], df['label'])

Processing batch 1/300
Raw response text: [
    {"ID": "0", "S": "P"},
    {"ID": "1", "S": "N"}
]
Processing batch 2/300
Raw response text: [
    {"ID": 2, "S": "N"},
    {"ID": 3, "S": "P"}
]
Processing batch 3/300
Raw response text: [
    {"ID": "4", "S": "P"},
    {"ID": "5", "S": "P"}
]
Processing batch 4/300
Raw response text: [
    {"ID": "6", "S": "N"},
    {"ID": "7", "S": "P"}
]
Processing batch 5/300
Raw response text: [
    {"ID": "8", "S": "P"},
    {"ID": "9", "S": "N"}
]
Processing batch 6/300
Raw response text: [
    {"ID": "10", "S": "N"},
    {"ID": "11", "S": "P"}
]
Processing batch 7/300
Raw response text: [
    {"ID": "12", "S": "P"},
    {"ID": "13", "S": "N"}
]
Processing batch 8/300
Raw response text: [
    {"ID": "14", "S": "P"},
    {"ID": "15", "S": "P"}
]
Processing batch 9/300
Raw response text: [
    {"ID": "16", "S": "N"},
    {"ID": "17", "S": "P"}
]
Processing batch 10/300
Raw response text: [
    {"ID": "18", "S": "N"},
    {"ID": "19", "S": "N"}
]
Pro

### **Result Z-COT**

In [ ]:
evaluate_classification(results_zero_cot['S'], df['label'])

Accuracy: 0.9483
Precision: 0.9495
Recall: 0.9483
F1 Score: 0.9483


# **Machine Learning**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
text = []
with open('/content/drive/MyDrive/SA/test/sents.txt', 'r', encoding='utf-8') as file:
    for line in file:
        text.append(line.strip())

label = []
with open('/content/drive/MyDrive/SA/test/sentiments.txt', 'r', encoding='utf-8') as file:
    for line in file:
        label.append(line.strip())

df_test = pd.DataFrame({'text': text, 'label': label})
df_test = (df_test[df_test['label'] != '1'])
for i in df_test.index:
  if df_test['label'][i] == '2':
    df_test['label'][i] = '1'
df_test.reset_index(drop=True, inplace=True)
df_test

,text,label
0,nói tiếng anh lưu loát .,1
1,giáo viên rất vui tính .,1
2,cô max có tâm .,1
3,"giảng bài thu hút , dí dỏm .",1
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...",0
...,...,...
2994,"giảng viên rất nhiệt tình , công bằng trong vi...",1
2995,"các slide khó hiểu , ngôn ngữ trong slide phức...",0
2996,giáo viên giảng dạy có tâm huyết .,1
2997,chia sẻ cho em nhiều điều hay .,1


In [ ]:
text = []
with open('/content/drive/MyDrive/SA/train/sents.txt', 'r', encoding='utf-8') as file:
    for line in file:
        text.append(line.strip())

label = []
with open('/content/drive/MyDrive/SA/train/sentiments.txt', 'r', encoding='utf-8') as file:
    for line in file:
        label.append(line.strip())

df_train = pd.DataFrame({'text': text, 'label': label})
df_train = (df_train[df_train['label'] != '1'])
for i in df_train.index:
  if df_train['label'][i] == '2':
    df_train['label'][i] = '1'
df_train.reset_index(drop=True, inplace=True)
df_train

,text,label
0,slide giáo trình đầy đủ .,1
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",1
2,đi học đầy đủ full điểm chuyên cần .,0
3,chưa áp dụng công nghệ thông tin và các thiết ...,0
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",1
...,...,...
10963,chỉ vì môn game mà em học hai lần mà không qua...,0
10964,em cảm ơn cô nhiều .,1
10965,giao bài tập quá nhiều .,0
10966,"giáo viên dạy dễ hiểu , nhiệt tình .",1


In [ ]:
df_train[df_train['label'] == '0'].count()

,0
text,5325
label,5325


In [ ]:
# Giả sử X (dữ liệu văn bản) và y (nhãn) đã được chuẩn bị
vectorizer = CountVectorizer()

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train = vectorizer.fit_transform(df_train['text'])
X_test = vectorizer.transform(df_test['text'])
y_train = df_train['label']
y_test = df_test['label']

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (10968, 2408)
X_test shape: (2999, 2408)
y_train shape: (10968,)
y_test shape: (2999,)


### **SVM**

In [ ]:
# Tạo và huấn luyện mô hình SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Dự đoán
y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
recall = recall_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
f1 = f1_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp

# In ra kết quả
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9340
Precision: 0.9357
Recall: 0.9340
F1 Score: 0.9340


### **LR**

In [ ]:
# Chuẩn hóa dữ liệu và tạo pipeline
scaler = StandardScaler(with_mean=False)  # with_mean=False để làm việc với ma trận sparse
lr_model = make_pipeline(scaler, LogisticRegression(C=0.2, max_iter=2000))

# Huấn luyện mô hình
lr_model.fit(X_train, y_train)

# Dự đoán
y_pred = lr_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
recall = recall_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
f1 = f1_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp

# In ra kết quả
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9213
Precision: 0.9213
Recall: 0.9213
F1 Score: 0.9213


### **RF**

In [ ]:
# Tạo và huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier(
    n_estimators=150,
    criterion='gini',
    max_depth=40,
)
rf_model.fit(X_train, y_train)

# Dự đoán
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
recall = recall_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
f1 = f1_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp

# In ra kết quả
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9210
Precision: 0.9234
Recall: 0.9210
F1 Score: 0.9210


### **DT**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Tạo và huấn luyện mô hình Decision Tree
dt_model = DecisionTreeClassifier(criterion='entropy', min_samples_split=10, random_state=226)
dt_model.fit(X_train, y_train)

# Dự đoán
y_pred_dt = dt_model.predict(X_test)

# Tính toán các chỉ số đánh giá
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt, average='weighted')  # average='weighted' cho nhiều lớp
recall_dt = recall_score(y_test, y_pred_dt, average='weighted')  # average='weighted' cho nhiều lớp
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')  # average='weighted' cho nhiều lớp

# In ra kết quả
print(f"Decision Tree Model")
print(f"Accuracy: {accuracy_dt:.4f}")
print(f"Precision: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")
print(f"F1 Score: {f1_dt:.4f}")


Decision Tree Model
Accuracy: 0.9026
Precision: 0.9029
Recall: 0.9026
F1 Score: 0.9027


### **NB**

In [ ]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

y_pred = nb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
recall = recall_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp
f1 = f1_score(y_test, y_pred, average='weighted')  # average='weighted' cho nhiều lớp

# In ra kết quả
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8860
Precision: 0.8886
Recall: 0.8860
F1 Score: 0.8861
